In [2]:
from cases import *

case = Debugger1


In [3]:
import numpy as np
from time import time 
np.random.seed(int(time()))

class Gene:
    fitness = 0
    def __init__(self, perm) -> None:
        self.perm = perm

class SA:
    def __init__(self, case: Case, T, alpha, mcn) -> None:
        self.case = case
        self.state = Gene(case.req.copy())
        np.random.shuffle(self.gene.perm)
        self.state.fitness = self.calculate_fitness(self.state)
        self.fittest = self.state

        self.T = T
        self.alpha = alpha
        self.N = mcn
        self.n = 0

    

    def iterate(self):
        neighbour = self.get_random_neighbour(self.state)
        fitness = self.calculate_fitness(neighbour)
